In [7]:
from dotenv import load_dotenv

load_dotenv("openai.env")

True

# 使用文件加载专用chains

-  https://smith.langchain.com/hub有大量实现好的chains,方便我们直接调用 


In [ ]:
! pip install langchainhub

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain import hub

# https://smith.langchain.com/hub
# Pull down prompt
prompt = hub.pull("rlm/text-to-sql")
# Initialize model
model = ChatOpenAI()

# 入参
# lambda x: 表示是一个匿名函数
inputs = {
    "table_info": lambda x: "customers",
    "input": lambda x: x["question"],
    "few_shot_examples": lambda x: "",
    "dialect": lambda x: "mysql",
}
sql_response = (
        inputs
        | prompt
        | model.bind(stop=["\nSQLResult:"])
        | StrOutputParser()
)

# Call with a given question
response = sql_response.invoke({"question": "How many customers are there?"})

print(response)

SQLQuery: SELECT COUNT(*) FROM customers;


# CustomChain
当通用链不满足的时候，可以自行构建来实现特定的目的
<hr>

In [25]:
from typing import List, Dict, Any, Optional
from langchain.callbacks.manager import (
    CallbackManagerForChainRun
)
from langchain.chains.base import Chain
from langchain.prompts.base import BasePromptTemplate
from langchain.base_language import BaseLanguageModel


class wiki_article_chain(Chain):
    """开发一个wiki文章生成器"""
    prompt: BasePromptTemplate
    llm: BaseLanguageModel
    out_key: str = "text"

    @property
    def input_keys(self) -> List[str]:
        """将返回Prompt所需的所有键"""
        return self.prompt.input_variables

    @property
    def output_keys(self) -> List[str]:
        """将始终返回text键"""
        return [self.out_key]

    @property
    def _chain_type(self) -> str:
        """链类型"""
        return "wiki_article_chain"

    def _call(
            self,
            inputs: Dict[str, Any],
            run_manager: Optional[CallbackManagerForChainRun] = None,
    ) -> Dict[str, Any]:
        """运行链"""
        prompt_value = self.prompt.format_prompt(**inputs)
        print("prompt_value:", prompt_value)
        response = self.llm.generate_prompt(
            [prompt_value], callbacks=run_manager.get_child() if run_manager else None
        )
        # print("response:", response)
        if run_manager:
            run_manager.on_text("wiki article is written")
        return {self.out_key: response.generations[0][0].text}


In [27]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

chain = wiki_article_chain(
    prompt=PromptTemplate(
        template="写一篇关于{topic}的维基百科形式的文章",
        input_variables=["topic"]
    ),
    llm=ChatOpenAI(temperature=0),
)

result = chain.invoke({"topic": "大头同学"})
print(result)

prompt_value: text='写一篇关于大头同学的维基百科形式的文章'
{'topic': '大头同学', 'text': '### 大头同学\n\n**大头同学**是中国著名网络文化人物之一，以其幽默风趣的表现和独特的创作风格而闻名于网络社区。他的作品涵盖了视频内容、文字创作以及社交媒体互动等多个领域，在中国互联网文化圈拥有广泛的影响力。\n\n---\n\n#### 个人资料\n\n- **姓名：** 大头同学\n- **出生日期：** 暂无确切资料\n- **国籍：** 中国\n- **职业：** 网络文化人物、视频创作者\n- **知名平台：** Bilibili、YouTube等\n\n#### 创作风格\n\n大头同学以幽默诙谐的表现方式和独特的创意脱颖而出。他常常在自己的视频中运用夸张的表情、搞笑的配音和生动的画面，将各种生活场景和话题进行有趣的演绎和解读。其创作涵盖了各种题材，包括但不限于日常生活、社会现象、影视评论等，且常常融入了大量的流行文化元素和网络用语，使得其作品更具有时代感和亲和力。\n\n#### 成就和影响\n\n大头同学的作品深受广大网友的喜爱，其视频在网络平台上拥有众多的粉丝和观众群体。他的创作不仅在中国，也在国际上获得了一定的知名度，吸引了许多海外观众的关注。除了在视频领域取得成功外，大头同学还活跃于各种社交媒体平台，与粉丝们进行互动，分享生活趣事和创作心得，进一步增强了他与粉丝之间的联系。\n\n#### 社会影响\n\n大头同学作为一名网络文化人物，不仅仅是娱乐大众的角色，他的作品也常常带有一定的思考和反思，对于社会现象、人生态度等方面进行了一定的探讨和表达。他的影响力不仅仅停留在娱乐领域，还扩展到了对年轻人价值观念的影响和引导，成为了许多年轻人心目中的偶像和榜样。\n\n#### 未来展望\n\n随着网络文化的不断发展和演变，大头同学作为其中的佼佼者之一，将继续为广大网友带来更多精彩的创作和内容。他的成功经验也将为更多的年轻人提供启示，激励他们在网络创作领域中追求自己的梦想，为中国网络文化的繁荣做出更多的贡献。\n\n---\n\n这是一份简要的关于大头同学的维基百科形式的文章，有什么需要补充或修改的吗？'}
